In [3]:
# !pip install pyecharts
# !pip install selenium
import numpy as np 
import matplotlib.pyplot as plt 
#!pip install yfinance
import yfinance as yf # https://pypi.org/project/yfinance/
import math
import random
import seaborn as sns
import datetime
import pandas as pd
from pyecharts.charts import Bar, Line
from pyecharts import options as opts

In [85]:
# get data by ticker-name, start-time & end-time
def get_df_data(ticker_name="AAPL", start_time="2022-01-01", end_time="2022-10-09"):
    df_data = yf.download(tickers=ticker_name, start=start_time, end=end_time) 
    df_data = df_data[ ["Open", "High", "Low", "Close", "Volume"] ]
    df_data['previous_Close'] = df_data['Close'].shift(1)
    df_data['daily_return'] = (df_data['Close']-df_data['previous_Close'])/df_data['previous_Close']
    return df_data

def draw_2lines(x, y1, tn1, y2, tn2, ls1=50, le1=160, ls2=50, le2=160):
    data_1 = y1 #np.sin(x)
    data_2 = y2 #np.cos(x)
    # Create Plot
    fig, ax1 = plt.subplots(figsize=(25, 10)) 
    ax1.set_xlabel('Time range') 
    ax1.set_ylabel(tn1, color = 'red') 
    ax1.plot(x, data_1, color = 'red', label=tn1) 
    ax1.tick_params(axis ='y', labelcolor = 'red') 
    ax1.set_ylim(ls1, le1)
    ax2 = ax1.twinx() 
    ax2.set_ylabel(tn2, color = 'blue') 
    ax2.plot(x, data_2, color = 'blue', label=tn2) 
    ax2.tick_params(axis ='y', labelcolor = 'blue') 
    ax2.set_ylim(ls2, le2)
    # show the graph
    plt.legend()
    plt.show()
    return 

"""
# data visualization
ls1=ls2=60 
le1=le2=100

y1, y2 = y1_open, y2_open
draw_2lines(x, y1, tn1, y2, tn2, ls1, le1, ls2, le2)
"""

def time_point_dic(time_list, point_close):
    dic = {}
    i = 0
    while i<len(time_list):
        dic[time_list[i]] = point_close[i]
        i += 1
    return dic

# 
def data_union(dic1, dic2):
    # add in complement time values
    time1 = list(dic1.keys())
    time2 = list(dic2.keys())
    time_union = sorted(list(set(time1+time2)))
    for t in time_union:
        if t not in dic1:
            dic1[t] = 0
        if t not in dic2:
            dic2[t] = 0
    return time_union

def bin_analysis(point_list, low_level=0.0, high_level=3.0, level_width=0.1):
    # low_level, high_level, level_width, 
    k_num = round((high_level-low_level)/level_width) + 1
    N = len(point_list)
    fre_dic = {}
    for k in range(k_num):
        fre_dic[round(k*level_width, 1)] = 0

    for v in point_list:
        k = round(v/level_width) * level_width
        if k>high_level:
            k = high_level
        k = round(k, 1)
        fre_dic[k] += 1

    cumulative = 0
    for k in range(k_num):
        k = round(k*level_width, 1)
        cumulative = cumulative + fre_dic[k]
        r = round(cumulative/N*100, 2)
        print( k, "\t", cumulative, "\t", r, "%" )
    return 

def find_previous_businessday(i, x, y):
    counter = i
    t = x[counter]
    p = y[counter]
    while p<0 and counter>0:
        t = x[counter]
        p = y[counter]
        counter = counter - 1
    #if counter!=i:
        #print( "\t reference stock in holiday \t", x[i], y[i], " -> ", t, p )
    return t, p

def find_next_businessday(i, j, x, y):
    counter = i
    point_list = []
    while counter<=j and counter<len(x):
        point_list.append( y[counter] )
        counter += 1
    if len(point_list)==0:
        return [0]
    return point_list

# 
def back_test(signal, in_days, x, 
              y1_open, y1_close, y1_low, y1_high, 
              y2_open, y2_close, y2_low, y2_high):
    other_info = []
    profit_list = []
    for i in range(0, len(x)-1):
            # reference stock info
        reference_t, reference_p = find_previous_businessday(i, x, y1_close)
            # target stock info
        target_t = x[i+1]
        target_open = y2_open[i+1]
        target_close = y2_close[i+1]
        target_low = y2_low[i+1]
        target_high = y2_high[i+1]
        if target_open<0:
            #print( target_t, "\t target stock in holiday" )
            continue
        #print( target_t, target_open, target_close, target_low, target_high, "\t", reference_t, reference_p )
            # possible profit
        if target_low<reference_p-signal:
            buy_point = min(target_open, reference_p-2)
            j = i+in_days
            high_list = find_next_businessday(i+2, j, x, y2_high)
            close_list = find_next_businessday(i+2, j, x, y2_close)
            sell_point = max( target_close, max(high_list) )
            profit = round( sell_point-buy_point, 2 )
            if profit<1:
                profit = round( close_list[-1]-buy_point, 2 )
            profit_list.append( profit )
            other_info.append( [reference_t, reference_p, target_t, target_open, target_close, target_low, target_high, buy_point, sell_point, profit] )
    return profit_list, other_info

def get_list(from_num=1.5, to_num=3.0, step_num=0.1):
    tmp_list = []
    num = from_num
    while num<=to_num:
        num = round(num, 1)
        tmp_list.append( num )
        num += step_num
    return tmp_list

def profit_analysis(profit_list, from_num=1.0, to_num=3.0, step_num=0.1):
    info = []
    N = len(profit_list)
    for profit_per_share in get_list(from_num, to_num, step_num):
        counter = 0
        for v in profit_list:
            if v >= profit_per_share:
                counter += 1
        r = round(counter / N * 100, 2)
        info.append( (profit_per_share, N, counter, r) )
    return info

#
# data_info, 
    # signal
        # in_days
            # profit_per_share, ratio
def data_by_signal(data_info, ll=2.0, hl=11):
    data, col_names = [], []
    for signal in range(hl):
        signal = ll + 0.1*signal
        signal = round(signal, 1)
        #
        info = data_info[signal]        
        for days in sorted( list(info.keys()) ):
            it = info[days]
            N = len(it)
            tmp_n = ["signal", "days"]
            tmp_v = [signal, days]   
            i = 0
            while i<N:
                tmp_n.append( "profit_" + str(it[i][0]) )
                tmp_v.append( it[i][1] )            
                i = i + 1
            data.append(tmp_v)
            col_names.append( tmp_n )
    col_names = col_names[0]
    #print(col_names)
    df = pd.DataFrame(data, columns=col_names)
    return df

def dict_lists(dic):
    x = sorted( list(dic.keys()) )
    y = []
    for t in x:
        y.append( dic[t] )
    return x, y

def cumulative_return(tmp_list):
    cr_list = []
    r = 1
    for v in tmp_list:
        r = r*(1+v)
        cr_list.append( round(r*100, 2) )
        #print(r)
    return cr_list

In [124]:
st, et = "2022-10-27", "2023-01-06"
print("time range:\t", st, "-", et)

df = pd.DataFrame()

# ('BABA', '9988.HK', '2022-11-01')
# ('BIDU', '9888.HK', '2022-10-25'): 
# ('NTES', '9999.HK'): 2022-10-26
# ('JD', '9618.HK'), ('MPNGY', '3690.HK'), ('TME', '0700.HK')
for it in [('BIDU', '9888.HK', '2022-10-25')]:
    tn1, tn2, st = it

    #tn1 = "BABA" # BABA, BIDU, JD, MPNGY, TME, NTES
    df_data1 = get_df_data(ticker_name=tn1, start_time=st, end_time=et)
    print("stock:\t", tn1, "\t", len(df_data1))

    #tn2 = "9988.HK" # 9988.HK, 9888.HK, 9618.HK, 3690.HK, 0700.HK, 9999.HK
    df_data2 = get_df_data(ticker_name=tn2, start_time=st, end_time=et)
    print("stock:\t", tn2, "\t", len(df_data2))

    # dict-type: key -> datetime, value -> point
    dic1_return = time_point_dic(list(df_data1.index), list(df_data1['daily_return']))
    dic2_return = time_point_dic(list(df_data2.index), list(df_data2['daily_return']))
    print( "2 stocks:\t")
    print( "\t", len(dic1_return) )
    print( "\t", len(dic2_return) )

    # data union, stock 1 may have some business-day when stock 2 in holiday, and vice versa
    time_union = data_union(dic1_return, dic2_return)
    print( "2 stocks:\t")
    print( "\t", len(dic1_return) )
    print( "\t", len(dic2_return) )

    #
    start_index, end_index = 1, 261
    x1, y1 = dict_lists(dic1_return)
    x1, y1 = x1[start_index : ], y1[start_index : ]
    cr_list1 = cumulative_return(y1)

    x2, y2 = dict_lists(dic2_return)
    x2, y2 = x2[start_index : ], y2[start_index : ]
    cr_list2 = cumulative_return(y2)
    assert len(x1)==len(x2)==len(y1)==len(y2)
    print( len(cr_list1), len(cr_list2) )

    """
    plt.subplots(figsize=(20, 6))
    plt.plot(x1[start_index : end_index], cr_list1[start_index : end_index], label="daily return " + str(tn1))
    plt.plot(x2[start_index : end_index], cr_list2[start_index : end_index], label="daily return " + str(tn2))
    plt.legend()
    plt.show()  # display
    """
    df.index = x1
    df[tn1] = cr_list1
    df[tn2] = cr_list2
#
MA = 3
tns = list(df.columns)
for col in tns:
    df[col+"_MA"] = df[col].rolling(MA).mean()
    df[col+"_MA"] = df[col+"_MA"].round(decimals = 2)
df

line_returns = (
    Line(init_opts=opts.InitOpts(width="1800px", height="960px"))      
    .set_global_opts(title_opts=opts.TitleOpts(title="stocks comparison: cumulative returns"), 
                     tooltip_opts=opts.TooltipOpts(trigger="axis"),
                    yaxis_opts=opts.AxisOpts(name='returns',splitline_opts=opts.SplitLineOpts(is_show=True),min_=80))# , subtitle="商店A中六樣商品數"
)

line_returns.add_xaxis( list(df.index)[MA:] )

tns = list(df.columns)
for col in tns:
    line_returns.add_yaxis(col+"", list(df[col])[MA:] )

line_returns.render("lines_returns01.html")

time range:	 2022-10-27 - 2023-01-06
[*********************100%***********************]  1 of 1 completed
stock:	 BIDU 	 50
[*********************100%***********************]  1 of 1 completed
stock:	 9888.HK 	 50
2 stocks:	
	 50
	 50
2 stocks:	
	 51
	 51
50 50


'C:\\Users\\Admin\\lines_returns01.html'